# DRP Afterburner for Super HATS (DASH)

starting hats tables:
diaobject
dia source (visit id, NOT MJD)
object
forced source (visit id, NOT MJD)

JIRA for this week: https://rubinobs.atlassian.net/browse/DM-48478

LINCC notebooks: https://github.com/lsst-sitcom/linccf/tree/main

https://github.com/LSSTScienceCollaborations/StackClub/tree/master

repo = /repo/main

collection = LSSTComCam/runs/DRP/DP1/w_2025_03/DM-48478

input collection: LSSTComCam/DP1/defaults 

step1 dataQuery: "instrument='LSSTComCam'”

In [1]:
# Generic python packages
import re
import pylab as plt
import gc
import numpy as np
import pandas as pd
import astropy.units as u

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

### Helper methods

In [2]:
def append_mag_and_magerr(df, flux_col_prefixes):
    """Calculate magnitudes and their errors for flux columns."""
    mag_cols = {}
    
    for prefix in flux_col_prefixes:
        # Magnitude
        flux = df[f"{prefix}Flux"]
        mag = u.nJy.to(u.ABmag, flux)
        mag_cols[f"{prefix}Mag"] = mag

        # Magnitude error, if flux error exists
        fluxErr_col = f"{prefix}FluxErr"
        if fluxErr_col in df.columns:
            fluxErr = df[fluxErr_col]
            upper_mag = u.nJy.to(u.ABmag, flux+fluxErr)
            lower_mag = u.nJy.to(u.ABmag, flux-fluxErr)
            magErr = -(upper_mag-lower_mag)/2
            mag_cols[f"{prefix}MagErr"] = magErr
        
    mag_df = pd.DataFrame(mag_cols, dtype=np.float64, index=df.index)
    return pd.concat([df, mag_df], axis=1)

def extract_flux_col_prefixes(df):
    """Extract the prefixes for the flux columns"""
    regex = r"^(.*?)(?=Flux)"
    flux_cols = df.columns[df.columns.str.contains("Flux") & ~df.columns.str.contains("flag")]
    flux_col_prefixes = set([re.match(regex,col).group(1) for col in flux_cols])
    return flux_col_prefixes

### Configure Butler

In [3]:
config = '/repo/main'
collections = 'LSSTComCam/runs/DRP/DP1/w_2025_03/DM-48478'
butler = dafButler.Butler(config, collections=collections)

### DIA Objects and Sources

In [4]:
diaObj_refs = butler.query_datasets("diaObjectTable_tract")

In [5]:
diaObj_refs[0]

DatasetRef(DatasetType('diaObjectTable_tract', {skymap, tract}, DataFrame), {skymap: 'lsst_cells_v1', tract: 2393}, run='LSSTComCam/runs/DRP/DP1/w_2025_03/DM-48478/20250119T135723Z', id=93514536-9c25-476d-9a05-d7c7181d820f)

In [6]:
tracts = [ref.dataId["tract"] for ref in diaObj_refs]
len(set(tracts))

28

In [7]:
# datasetType = 'objectTable_tract'
datasetType='diaObjectTable_tract'
obj = butler.get(datasetType, dataId=diaObj_refs[0].dataId)
print(f"Retrieved catalog of {len(obj)} objects.")

Retrieved catalog of 758 objects.


In [8]:
obj.columns

Index(['ra', 'dec', 'nDiaSources', 'radecMjdTai', 'r_psfFluxLinearSlope',
       'r_psfFluxLinearIntercept', 'r_psfFluxMAD', 'r_psfFluxMaxSlope',
       'r_psfFluxErrMean', 'r_psfFluxMean',
       ...
       'y_psfFluxPercentile05', 'y_psfFluxPercentile25',
       'y_psfFluxPercentile50', 'y_psfFluxPercentile75',
       'y_psfFluxPercentile95', 'y_psfFluxSigma', 'y_scienceFluxSigma',
       'y_psfFluxSkew', 'y_psfFluxChi2', 'y_psfFluxStetsonJ'],
      dtype='object', length=114)

In [9]:
dataId = {'skymap': 'lsst_cells_v1', 'tract': 2393}
# datasetType = 'forcedSourceTable_tract'
datasetType='diaSourceTable_tract'
diaSrc = butler.get(datasetType, dataId=dataId)
print(f"Retrieved catalog of {len(diaSrc)} sources.")

Retrieved catalog of 1141 sources.


In [10]:
diaSrc.columns

Index(['visit', 'detector', 'band', 'diaObjectId', 'ssObjectId',
       'parentDiaSourceId', 'midpointMjdTai', 'bboxSize', 'time_processed',
       'ra', 'dec', 'raErr', 'decErr', 'ra_dec_Cov', 'x', 'y', 'xErr', 'yErr',
       'apFlux', 'apFluxErr', 'snr', 'psfFlux', 'psfFluxErr', 'psfChi2',
       'psfNdata', 'trailFlux', 'trailRa', 'trailDec', 'trailLength',
       'trailAngle', 'dipoleMeanFlux', 'dipoleMeanFluxErr', 'dipoleFluxDiff',
       'dipoleFluxDiffErr', 'dipoleLength', 'dipoleAngle', 'dipoleChi2',
       'isDipole', 'dipoleFitAttempted', 'dipoleNdata', 'scienceFlux',
       'scienceFluxErr', 'ixx', 'iyy', 'ixy', 'ixxPSF', 'iyyPSF', 'ixyPSF',
       'extendedness', 'reliability', 'pixelFlags', 'pixelFlags_offimage',
       'pixelFlags_edge', 'pixelFlags_interpolated', 'pixelFlags_saturated',
       'pixelFlags_cr', 'pixelFlags_bad', 'pixelFlags_suspect',
       'pixelFlags_interpolatedCenter', 'pixelFlags_saturatedCenter',
       'pixelFlags_crCenter', 'pixelFlags_suspectCent

In [11]:
flux_col_prefixes = extract_flux_col_prefixes(diaSrc)
# dipoleFluxDiff,dipoleFluxDiffErr do not work
flux_col_prefixes.remove("dipole")
flux_col_prefixes

{'ap', 'dipoleMean', 'psf', 'science', 'trail'}

In [ ]:
diaSrc_with_mags = append_mag_and_magerr(diaSrc, flux_col_prefixes)
diaSrc_with_mags

### Objects

In [13]:
objs_refs = butler.query_datasets("objectTable")

In [14]:
objs_tracts = [ref.dataId["tract"] for ref in objs_refs]
len(set(objs_tracts))

28

In [15]:
objs_tracts[0]

453

In [16]:
datasetType = 'objectTable'
objs = butler.get(datasetType, dataId=objs_refs[0].dataId)
print(f"Retrieved catalog of {len(objs)} objects.")

Retrieved catalog of 1113 objects.


In [17]:
cols_per_band = []
for band in list("ugrizy"):
    for flux_name in ["psf","kron"]:
        cols_per_band.extend([f"{band}_{flux_name}Flux", f"{band}_{flux_name}FluxErr"])
    cols_per_band.append(f"{band}_kronRad")

In [18]:
obj_default_columns = [
    "refFwhm",
    "shape_flag",
    "sky_object",
    "parentObjectId",
    "detect_isPrimary",
    "x",
    "y",
    "xErr",
    "yErr",
    "shape_yy", 
    "shape_xx", 
    "shape_xy", 
    "coord_ra",
    "coord_dec", 
    "coord_raErr", 
    "coord_decErr",
    "tract", 
    "patch",
    "detect_isIsolated"
] + cols_per_band

obj_default_columns

['refFwhm',
 'shape_flag',
 'sky_object',
 'parentObjectId',
 'detect_isPrimary',
 'x',
 'y',
 'xErr',
 'yErr',
 'shape_yy',
 'shape_xx',
 'shape_xy',
 'coord_ra',
 'coord_dec',
 'coord_raErr',
 'coord_decErr',
 'tract',
 'patch',
 'detect_isIsolated',
 'u_psfFlux',
 'u_psfFluxErr',
 'u_kronFlux',
 'u_kronFluxErr',
 'u_kronRad',
 'g_psfFlux',
 'g_psfFluxErr',
 'g_kronFlux',
 'g_kronFluxErr',
 'g_kronRad',
 'r_psfFlux',
 'r_psfFluxErr',
 'r_kronFlux',
 'r_kronFluxErr',
 'r_kronRad',
 'i_psfFlux',
 'i_psfFluxErr',
 'i_kronFlux',
 'i_kronFluxErr',
 'i_kronRad',
 'z_psfFlux',
 'z_psfFluxErr',
 'z_kronFlux',
 'z_kronFluxErr',
 'z_kronRad',
 'y_psfFlux',
 'y_psfFluxErr',
 'y_kronFlux',
 'y_kronFluxErr',
 'y_kronRad']

In [ ]:
objs = objs[obj_default_columns]
objs

In [20]:
flux_col_prefixes = extract_flux_col_prefixes(objs)
flux_col_prefixes

{'g_kron',
 'g_psf',
 'i_kron',
 'i_psf',
 'r_kron',
 'r_psf',
 'u_kron',
 'u_psf',
 'y_kron',
 'y_psf',
 'z_kron',
 'z_psf'}

In [ ]:
objs_with_mags = append_mag_and_magerr(objs, flux_col_prefixes)
objs_with_mags

### Forced Sources

In [22]:
fSrc_refs = butler.query_datasets("forcedSourceTable")

In [23]:
fSrctracts = [ref.dataId["tract"] for ref in fSrc_refs]
len(set(fSrctracts))

28

In [24]:
fSrc_refs[0]

DatasetRef(DatasetType('forcedSourceTable', {skymap, tract, patch}, DataFrame), {skymap: 'lsst_cells_v1', tract: 10464, patch: 75}, run='LSSTComCam/runs/DRP/DP1/w_2025_03/DM-48478/20250119T135723Z', id=025a6aba-3575-4101-abc4-7be4397078e3)

In [25]:
datasetType = 'forcedSourceTable'
fSrc = butler.get(datasetType, dataId=fSrc_refs[0].dataId)
print(f"Retrieved catalog of {len(fSrc)} sources.")

Retrieved catalog of 94981 sources.


In [ ]:
fSrc

In [27]:
visits = butler.query_datasets("visitTable")

In [28]:
visits[0]

DatasetRef(DatasetType('visitTable', {instrument}, DataFrame), {instrument: 'LSSTComCam'}, run='LSSTComCam/runs/DRP/DP1/w_2025_03/DM-48478/20250117T153854Z', id=1b9f197f-cf7d-4783-af81-126b2477ed6f)

In [29]:
datasetType = 'visitTable'
visits = butler.get(datasetType, dataId={'instrument': 'LSSTComCam'})
print(f"Retrieved catalog of {len(visits)} visits.")

Retrieved catalog of 1857 visits.


In [ ]:
visits

In [31]:
visit_map = visits[["expMidptMJD"]].T.to_dict('records')[0]
mjds = list(map(lambda x: visit_map.get(x, 0.0), fSrc["visit"]))
fSrc["midpointMJDTai"] = pd.Series(mjds, index=fSrc.index)

In [ ]:
fSrc

In [33]:
flux_col_prefixes = extract_flux_col_prefixes(fSrc)
flux_col_prefixes

{'localBackground_inst', 'psf', 'psfDiff'}

In [ ]:
fSrc_with_mags = append_mag_and_magerr(fSrc, flux_col_prefixes)
fSrc_with_mags